In [1]:
!nvidia-smi

Sun Jul 23 13:12:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

In [4]:
#데이터 불러오기
data_cov = pd.read_csv('/content/drive/MyDrive/데이터분석공모전스터디/covtype.csv')
data_cov['Cover_Type'].value_counts()
#class1 == 3, class0 == 4
data1= data_cov[data_cov['Cover_Type'] == 3]
data0 = data_cov[data_cov['Cover_Type'] == 4]

df = pd.concat([data1,data0])

dataset=df.sample(frac=1).reset_index(drop=True) #샘플 섞어주기
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2015,43,30,90,32,216,210,160,65,633,...,0,0,0,0,0,0,0,0,0,3
1,2059,185,28,67,26,443,212,245,147,659,...,0,0,0,0,0,0,0,0,0,3
2,2222,357,9,30,1,591,206,224,156,1752,...,0,0,0,0,0,0,0,0,0,3
3,2428,349,24,495,238,1230,167,191,157,371,...,0,0,0,0,0,0,0,0,0,3
4,2384,88,6,85,14,1159,230,229,133,726,...,0,0,0,0,0,0,0,0,0,3


In [7]:
dataset['Cover_Type'].replace([3, 4],[1, 0],inplace=True)

In [8]:
dataset['Cover_Type'].value_counts()

1    35754
0     2747
Name: Cover_Type, dtype: int64

In [9]:
X = dataset.drop('Cover_Type', axis =1)
y = dataset['Cover_Type']

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

X_scaled = scaler.transform(X)

scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
scaled_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,-1.869260,-1.223482,1.037700,-0.781408,-0.488898,-1.208869,0.153858,-2.035531,-1.388605,-0.523291,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.645360,0.107070,0.816671,-0.940593,-0.590294,-0.830795,0.203521,1.059085,0.164816,-0.473682,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.815912,1.718724,-1.283109,-1.196671,-1.012778,-0.584297,0.054531,0.294533,0.335314,1.611789,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.232348,1.643763,0.374612,2.021616,2.992365,0.479973,-0.913904,-0.906907,0.354258,-1.023193,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.008448,-0.801828,-1.614653,-0.816014,-0.793086,0.361721,0.650491,0.476569,-0.100402,-0.345844,...,0.0,-0.052543,-0.011397,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
dataset = pd.concat([scaled_df,y], axis = 1)

<ipython-input-11-071fbbc85173>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  dataset = pd.concat([scaled_df,y],1)


In [12]:
# 신규 (테스트) 데이터 생성 Train / Validation / Test
data = dataset.sample(frac=0.6, random_state=786)
data_test = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_test.shape))

Data for Modeling: (23101, 55)
Unseen Data For Predictions: (15400, 55)


In [13]:
train, val = train_test_split(data, test_size = 0.3, random_state = 9492)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gpu를 사용할 수 있으면 사용하고, 아니면 cpu를 사용하겠다

train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
val_x = val.iloc[:,:-1]
val_y = val.iloc[:,-1]

trn_y  = torch.FloatTensor(train_y.values)
trn_X  = torch.FloatTensor(train_x.values)
val_y  = torch.FloatTensor(val_y.values)
val_X  = torch.FloatTensor(val_x.values)


train = TensorDataset(trn_X, trn_y)
val  = TensorDataset(val_X, val_y)


batch_size = 64

train_kwargs = {'dataset': train,
                'batch_size': batch_size,
               'shuffle': True}

test_kwargs = {'dataset': val,
                'batch_size': batch_size,
              'shuffle': False}


train_loader = torch.utils.data.DataLoader(**train_kwargs)
val_loader  = torch.utils.data.DataLoader(**test_kwargs)

In [24]:
next(iter(train_loader))[1]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1.])

In [20]:
#DataLoader -> for 문과 함께 동작

In [ ]:
#gpu 사용
# use_cuda = torch.cuda.is_available()

In [25]:
class DNN(nn.Module): #클래스 = 붕어빵 틀, 붕어를 여러개 만들 수 있음
# 붕어빵 만들 때 들어가는 재료: 팥, 밀가루
# 붕어빵 틀 -> 팥, 밀가루 입력, 불 or 전기 -> self
# DNN -> ?, ? 입력

    def __init__(self, input_size, output_size, drop_rate):
        super(DNN, self).__init__() #nn.Module 상속 받음

        layer1 = [nn.Linear(input_size, 48),
                 nn.BatchNorm1d(48),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer2 = [nn.Linear(48, 24),
                 nn.BatchNorm1d(24),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer3 = [nn.Linear(24, 12),
                 nn.BatchNorm1d(12),
                 nn.Dropout(drop_rate),
                 nn.ReLU()]

        layer4 = [nn.Linear(12, output_size),
                 nn.Sigmoid()]

        self.fisrt_layer = nn.Sequential(*layer1)
        self.second_layer = nn.Sequential(*layer2)
        self.third_layer = nn.Sequential(*layer3)
        self.last_layer = nn.Sequential(*layer4)

    def forward(self, x):
        x = self.fisrt_layer(x)
        x = self.second_layer(x)
        x = self.third_layer(x)
        out = self.last_layer(x)

        return out

In [28]:
model_kwargs = {'drop_rate': 0.56,
              'input_size': 54,
               'output_size': 1}

training_kwargs = {'learning_rate': 1e-3,
              'epoch': 1}

model = DNN(**model_kwargs).to(device)

criterion = nn.BCELoss() #binary cross entropy

optimizer = optim.Adam(model.parameters(), lr=training_kwargs['learning_rate'])

num_epochs = training_kwargs['epoch']

In [29]:
print(model)

DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [ ]:
print('Neural network')
print(model)

Neural network
DNN(
  (fisrt_layer): Sequential(
    (0): Linear(in_features=54, out_features=48, bias=True)
    (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (second_layer): Sequential(
    (0): Linear(in_features=48, out_features=24, bias=True)
    (1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (third_layer): Sequential(
    (0): Linear(in_features=24, out_features=12, bias=True)
    (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.56, inplace=False)
    (3): ReLU()
  )
  (last_layer): Sequential(
    (0): Linear(in_features=12, out_features=1, bias=True)
    (1): Sigmoid()
  )
)


In [32]:
import copy
import warnings
warnings.filterwarnings("ignore")
#loss
criterion = nn.BCELoss()

def train(model, device, train_loader, optimizer, epoch):
    global criterion
    model.train() # batchnorm, dropout -> 항상 활성화 된 상태
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #초기화
        output = model(data) #output: 64 x 1
        output = output.squeeze(1) #output: 64
        loss = criterion(output, target)
        loss.backward()
        optimizer.step() #실행 64개 데이터를 가지고 모델에 넣고 weight를 업데이트 함
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    global criterion
    model.eval() # batchnorm, dropout -> 비 활성화된 상태
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for batch_idx, (data, target) in enumerate (val_loader):
            valid_loss = {'loss_val' : []}

            data, target = data.to(device), target.to(device)
            outputs = model(data)
            outputs = outputs.squeeze()

            loss_val = criterion(outputs, target)
            valid_loss['loss_val'].append(loss_val.item())

            outputs = torch.flatten(outputs).cpu().detach().numpy()
            target = torch.flatten(target).cpu().detach().numpy()
            predicted = np.where(outputs>0.5, 1, 0)

            n_samples += len(target)
            n_correct += (predicted == target).sum().item()
            vl_mean = np.array(valid_loss['loss_val']).mean()

            if batch_idx % 100 == 0:
                print('Batch mean Validation Loss: {:.6f}'.format(vl_mean))

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 validation set: {acc} %')
    return acc

In [34]:
# OUTPUT_PATH = '/home/user/Code/JE/informs'
best_acc = -1
best_model = None
for epoch in range(0, 2):
    train(model, device, train_loader, optimizer, epoch)
    acc = test()
    if acc > best_acc:
        best_acc = acc
        best_model = copy.deepcopy(model)
        print(f'new best acc={best_acc}')
    else:
        print(f'current acc={acc}, prev_best_acc={best_acc}')

# state = {
#     'model_state': best_model.state_dict(),
#     'test_acc': best_acc
# }
# torch.save(state, OUTPUT_PATH+'/practice_nn.pt')  #훈련 끝나고 모델을 저장
print('best model saved')

Train Epoch: 0 [0/16170 (0%)]	Loss: 0.247297
Train Epoch: 0 [6400/16170 (40%)]	Loss: 0.094964
Train Epoch: 0 [12800/16170 (79%)]	Loss: 0.166208
Batch mean Validation Loss: 0.093558
Batch mean Validation Loss: 0.095257
Accuracy of the network on the 10000 validation set: 92.33876785456644 %
new best acc=92.33876785456644
Train Epoch: 1 [0/16170 (0%)]	Loss: 0.129167
Train Epoch: 1 [6400/16170 (40%)]	Loss: 0.148158
Train Epoch: 1 [12800/16170 (79%)]	Loss: 0.115853
Batch mean Validation Loss: 0.090517
Batch mean Validation Loss: 0.095185
Accuracy of the network on the 10000 validation set: 92.30991198961189 %
current acc=92.30991198961189, prev_best_acc=92.33876785456644
best model saved


In [35]:
#new data inference

x = torch.normal(mean=6.9671e-06, std=0.4746, size=(1,54))
out = model(x.to(device))
out = torch.flatten(out).cpu().detach().numpy()
predicted = np.where(out>0.5, 1, 0)

In [36]:
x

tensor([[-1.0190,  0.1947,  0.4508,  0.3646,  0.3263,  0.1591,  0.1037, -1.1476,
         -0.6296,  0.8667, -0.3514, -0.0444,  0.6086, -0.7038, -0.4811, -0.0598,
          1.1431,  0.6484, -0.2312,  0.2580, -1.4168,  0.1509, -0.2521,  0.0451,
         -0.1366, -0.0978,  0.4961, -0.9659,  0.8949,  0.0740, -0.0037, -0.0203,
          0.1706, -0.3122,  0.2594, -0.1657, -0.1064,  0.8692, -0.2143, -0.6020,
         -0.4953,  0.4740, -0.9510,  0.4985, -0.5314, -0.1800,  0.0250, -0.4946,
          0.3013, -0.2030, -0.2170,  0.1093, -0.9191, -0.4835]])

In [37]:
print('예측 클래스:', predicted)

예측 클래스: [1]


## [HW] 3-layer neural network 를 구현하고 학습하세요(batch norm, drop out, activation function 자유 구성)

In [ ]:
class 3_layer_nn(nn.Module):

    def __init__(self):
        super(3_layer_nn, self).__init__()

        '''여기에 코드를 작성하세요'''

    def forward(self, x):
        '''여기에 코드를 작성하세요'''

        return